# Module 3 - Web APIs
---

## Introduction

- Back to [Table of Contents](#Table-of-Contents)

In this module, we introduce two general ways that one can retrieve data from data sources on the Internet: APIs and web scraping. We've already covered web scraping, and given the messiness of that subject you may find yourself really appreciating the relative simplicity of using APIs - once you grasp the concepts.

API stands for "Application Programming Interface". An API is an agreed upon way for one computer program to interact with another computer program.  There are many different kinds of APIs.  Some facilitate interaction between computers over the Internet, some do not. In fact, the Python module SQL Alchemy that we used yesterday is a type of API for talking to databases - you'll see more on APIs for databases with SQL Alchemy later in this course.

For this session, we focus on web APIs over HTTP that let a user query and retrieve data over the Internet.  This type of API documents an agreed-upon structure and content of requests and responses a program can use to interact with a system.  As long as your code adheres to a system's API, it should be able to reliabily request and receive data from the system.

Below, we show how to make network API requests using HTTP(S).

## Learning Objectives

- Back to [Table of Contents](#Table-of-Contents)

** Learning objectives:**

- **Become familiar with different types of APIs.** Includes GET- and POST- based HTTP APIs, different formats of request bodies for POST-based APIs (form inputs, arbitrary JSON and XML, and then formalized dialects of each like SOAP), and how to learn a given API.
- **Learn the tools used to interact with network-based APIs.** Understand the tools for talking directly with APIs over HTTP connection, introduce libraries that abstract the details of the API and present a simplified programmatic interface, and then understand how to choose a tool.

# Table of Contents
- [Introduction](#Introduction)

    - [Learning Objectives](#Learning-Objectives)
    - [Topics](#Topics)

- [Setup - Load Python packages](#Setup---Load-Python-packages)
- [Using APIs](#Using-APIs)
    * [RESTful APIs](#RESTful-APIs)
    * [OTP description](#OpenTripPlanner)
    * [Index API](#Index-API)
    * [JSON](#JSON)
    * [API Documentation](#API-Documentation)
    * [Python Dictionary](#Python-Dictionary)
    * [Routing API](#Routing-API)
        * [Scraping Useful Chicago Data](#Scraping-Useful-Chicago-Data)
        * [Geocoding](#Geocoding)
        * [Planner Resource Syntax](#Planner-Resource-Syntax)
        * [Incorporating Databases](#Incorporating-Databases)
    * [Isochrone API](#Isochrone-API)

## Topics

- Back to [Table of Contents](#Table-of-Contents)

Outline of topics covered in this notebook:

- Making raw HTTP API requests
- Using pre-packaged API client libraries
- Practical considerations - Knowing API rules and coding to follow them, and performance
- Example: OpenTripPlanner

# Setup - Load Python packages

- back to [Table of Contents](#Table-of-Contents)

In [160]:
## import Python packages ##
import time # to convert time as needed and report how long some functions take

# interacting with websites and web-APIs
import requests # easy way to interact with web sites and services
import json # read/write JavaScript Object Notation (JSON)
from bs4 import BeautifulSoup

# data manipulation
import pandas as pd # easy data manipulation
from pandas import DataFrame
import numpy as np
import math
# import geopandas as gpd # geographic data manipulation
# from geopandas.tools import sjoin, overlay # spatial join and overlay functions
# from shapely.geometry import Point, LineString # to create lines from a list of points

# visualization
import matplotlib as mplib
import matplotlib.pyplot as plt # visualization package

# so images get plotted in the notebook
%matplotlib inline

In [3]:
print("Package versions")
print("requests: {}".format(requests.__version__))
print("json: {}".format(json.__version__))
print("pandas: {}".format(pd.__version__))
print("matplotlib: {}".format(mplib.__version__))

Package versions
requests: 2.18.4
json: 2.0.9
pandas: 0.20.3
matplotlib: 2.0.2


# Using APIs

- Back to the [Table of Contents](#Table-of-Contents)

## API overview
+ In general: APIs ([Application Programming Interfaces](https://en.wikipedia.org/wiki/Application_programming_interface)) are "set[s] of subroutine definitions, protocols, and tools for building software and applications. A good API makes it easier to develop a program by providing all the building blocks, which are then put together by the programmer."
+ Here we're looking at a **web-API**, a specific type of API which makes it easier to interact with some aspect of a website. In this course, we'll be using APIs to gether data in an automated way - like grabbing a bunch of prior tweets from Twitter. More generally, APIs can also be used to interact with websites in any way the API is designed to. For instance, you can post and delete tweets with Twitter's API, too.

We're going to start getting our hands dirty with the various APIs of the OpenTrip Planner (OTP) software.

## RESTful APIs

The OTP APIs are what is called "[RESTful](https://en.wikipedia.org/wiki/Representational_state_transfer)" web services. REST stands for REpresentational State Transfer, but don't worry about the acronym so much as the idea. RESTful services adhere to a [series of requirements](https://www.restapitutorial/whatisrest.html) that enable them to be consistent, scalable, reliable, and relatively simple. RESTful APIs allow you to access a pre-defined set of operations through HTTP(S) requests. REST is fantastic because, in part, if you can generate the right URL, you'll always get the right response (this was not always the case with SOAP - the predecessor to REST).

To use a RESTful API, we'll need to understand (1) how to properly format the request and (2) how to manage and make use of the response from the API. Below we will walk through these concepts while using some of OTP's web services.

## PatentsView
[PatentsView](http://http://www.patentsview.org/api/doc.html) is an a USPTO documnetation service that offers logitudinal data and metadata on patents, inventors, companies, and geographic locations. The API is organized around 7 endpoints. Here we'll explore:
1. [Patents Endpoint](#Patents-Endpoint) - provides information about the different patents including application date, inventor, assignee;
1. [Inventor Endpoint](#Inventors-Endpoint) - provides information about the different inventor including number of patents, location(s), co-authors;
1. [Assignee Endpoint](#Assignee-Endpoint) - provides information about the different assignees including number of patents, location(s).

In [96]:
# base URL where OTP is installed - ennd point "routers/" simply lists information about the router
base_url = "http://www.patentsview.org/api/"

# Check connection (should be 200):
response = requests.get(base_url)
print(response.status_code)

200


### Patents Endpoint

- Back to the [Table of Contents](#Table-of-Contents)

The Patents Endpoint of the PatentsView API provides access to general information about the different patents in the database. A full list of the fields and options is provided [here](http://www.patentsview.org/api/patent.html).

Below, we add the `patents` endpoint to our base URL in order to explore the Patents API. We can then search for patents matching a query (`q`), return given fields (`f`), sorted by a sort-field (`s`), and using provided options (`o`).

The combination of the endpoint, query, fields, sort, and options will return the requested patent information.

In [102]:
# Define the Endpoint as Patents:
endpoint = 'patents/'

In [106]:
# Let's search for patents 7861317 and 7965342 return inventor name:
# Query:
query = 'query?q={"_or":[{"patent_number":"7861317"},{"patent_number":"7965342"}]}'
# Fields:
fields = '&f=["inventor_first_name","inventor_last_name", "patent_title"]'
# Sort:
sort = ''
# Options:
options = ''

url = base_url + endpoint + query + fields + sort + options

# get query response from API
response = requests.get(url)
print(response.text)

{"patents":[{"patent_title":"Nose cover","inventors":[{"inventor_first_name":"Robert Gregory","inventor_last_name":"Beliveau","inventor_key_id":"8781157"}]},{"patent_title":"Video image display device and video image display method","inventors":[{"inventor_first_name":"Tatsuhiko","inventor_last_name":"Nobori","inventor_key_id":"9057939"}]}],"count":2,"total_patent_count":2}


## JSON

- Back to the [Table of Contents](#Table-of-Contents)

[JSON](www.json.org) is a common non-tabular data format often used by services and software on the internet. We'll introduce JSON slowly, but it's helpful to know that it is oriented around the idea of `key-value` pairs. The `keys` refer to information about the data while the `values` is the data itself. For instance, if you were to translate a tabular dataset into JSON, the column names (and possibly row names/numbers) would become `keys`, while the data in the cells would become `values`. Our JSON response is a `dict` (dictionary), a Python object that we will describe below.

In [107]:
# Convert text to a Python object using the 'json' package
response = json.loads(response.text)

# And now we have a Python list:
print(type(response))
response

<class 'dict'>


{'count': 2,
 'patents': [{'inventors': [{'inventor_first_name': 'Robert Gregory',
     'inventor_key_id': '8781157',
     'inventor_last_name': 'Beliveau'}],
   'patent_title': 'Nose cover'},
  {'inventors': [{'inventor_first_name': 'Tatsuhiko',
     'inventor_key_id': '9057939',
     'inventor_last_name': 'Nobori'}],
   'patent_title': 'Video image display device and video image display method'}],
 'total_patent_count': 2}

## Python Dictionary

- Back to the [Table of Contents](#Table-of-Contents)

The [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), or dict for short, is a common type of python object used to store sets of key-value pairs. Sound familiar? It should! Dictionaries are python's internal counterpart to JSON data.

Here, we'll learn to grab data from within a dict by using the key name, following this syntax: `dict['key']`

In [114]:
patents = response['patents']
patents

[{'inventors': [{'inventor_first_name': 'Robert Gregory',
    'inventor_key_id': '8781157',
    'inventor_last_name': 'Beliveau'}],
  'patent_title': 'Nose cover'},
 {'inventors': [{'inventor_first_name': 'Tatsuhiko',
    'inventor_key_id': '9057939',
    'inventor_last_name': 'Nobori'}],
  'patent_title': 'Video image display device and video image display method'}]

In [115]:
print(patents[0]['patent_title'])

Nose cover


### Another Example: Querying all patents by Steve Jobs

In [116]:
# Let's search for all patent by Steve Jobs:
# Query:
query = 'query?q={"_and":[{"inventor_last_name":"Jobs"},{"inventor_first_name":"Steve P."}]}'
# Fields:
fields = '&f=["inventor_first_name","inventor_last_name", "patent_title"]'
# Sort:
sort = '&s=[{"patent_date":"desc"}]'
# Options:
options = ''

url = base_url + endpoint + query + fields + sort + options

# get query response from API
response = requests.get(url)
# Convert response to text
response = json.loads(response.text)

# Grab Patent Title
for i in range(0, len(response['patents'])):
    print(response['patents'][i]['patent_title'])

Dynamically changing appearances for user interface elements during drag-and-drop operations
User interface for providing consolidation and access
Electronic device
Modal change based on orientation of a portable multifunction device
Electronic device
Electronic device
Audio file interface
Computer
Mouse
Remote motion control using a wireless mobile device
Persistent group of media items for a media device
Electronic device
Portable display device with graphical user interface
Portable computer
Portable display device
Portable multifunction device, method, and graphical user interface supporting user navigations of graphical objects on a touch screen display
Touch screen device, method, and graphical user interface for customizing display of content category icons
Electronic device
Graphical user interface and methods of use thereof in a multimedia player
Portable display device with graphical user interface
Electronic device with graphical user interface
Portable display device
Power 

### Assignee Endpoint

- Back to the [Table of Contents](#Table-of-Contents)

The Assignee Endpoint of the PatentsView API provides access to general information about a given inventor in the database. A full list of the fields and options is provided [here](http://www.patentsview.org/api/assignee.html).

Below, we add the `assignee` endpoint to our base URL in order to explore the Patents API. We can then search for patents matching a query (`q`), return given fields (`f`), sorted by a sort-field (`s`), and using provided options (`o`).

The combination of the endpoint, query, fields, sort, and options will return the requested patent information.

In [74]:
# Define the Endpoint as Patents:
endpoint = 'assignees/'

In [77]:
# Let's search for assignee Apple, return inventor name:
# Query:
query = 'query?q={"assignee_organization":"Apple Inc."}'
# Fields:
fields = '&f=["assignee_organization","assignee_total_num_patents"]'
# Sort:
sort = ''
# Options:
options = ''

url = base_url + endpoint + query + fields + sort + options

# get query response from API
response = requests.get(url)
# Convert response to text
response = json.loads(response.text)

response

{'assignees': [{'assignee_organization': 'Apple Inc.',
   'assignee_total_num_patents': '14444'}],
 'count': 1,
 'total_assignee_count': 1}

### Inventor Endpoint

- Back to the [Table of Contents](#Table-of-Contents)

The Inventor Endpoint of the PatentsView API provides access to general information about a given inventor in the database. A full list of the fields and options is provided [here](http://www.patentsview.org/api/assignee.html).

Below, we add the `assignee` endpoint to our base URL in order to explore the Patents API. We can then search for patents matching a query (`q`), return given fields (`f`), sorted by a sort-field (`s`), and using provided options (`o`).

The combination of the endpoint, query, fields, sort, and options will return the requested patent information.

In [74]:
# Define the Endpoint as Patents:
endpoint = 'inventors/'

In [136]:
# Let's search for Steve Jobs patents, return all co-author names:
# Query:
query = 'query?q={"_and":[{"inventor_last_name":"Jobs"},{"inventor_first_name":"Steve P."}]}'
# Fields:
fields = '&f=["inventor_last_name","inventor_first_name","inventor_total_num_patents"]'
# Sort:
sort = ''
# Options:
options = '&o={"per_page":1000}'

url = base_url + endpoint + query + fields + sort + options

# get query response from API
response = requests.get(url)
# Convert response to text
response = json.loads(response.text)
patents = response['patents']

co_autors = []
for i in range(0, len(patents)):
    inventors = patents[i]['inventors']
    for j in range(0, len(inventors)):
        inventor = inventors[j]['inventor_first_name']+" "+inventors[j]['inventor_last_name']
        if (inventor!="Steve P. Jobs")&(inventor not in co_autors):
            co_autors.append(inventor)
print(co_autors)

['Jean-Marie Hullot', 'Christopher Matthew Franklin', 'Daniel N. Crow', 'Cary Dean', 'Elizabeth Ann Dykstra-Erickson', 'J. Peter Hoddie', 'Timothy E. Wasko', 'Donald Lindsay', 'Tim Wasko', 'Robert B. Brownell, Jr.', 'Sung Gwan Kim', 'Anwyl M. McDonald', 'Russell C. Mead, Jr.', 'Arturo Meuniot', 'Jesus Beltran', 'Bartley K. Andre', 'Christopher J. Stringer', 'Daniel J. Riccio, Jr.', 'Jonathan P. Ive', 'Michael D. Hillman', 'Frank Chee-Da Tsai', 'Michael D. McBroom', 'Daniel L. McBroom', 'Brian Sudderth', 'Arthur S. Brigham', 'Joel F. Jensen', 'Scott Nelson', 'Karl Backus', 'Rosa Sheng', 'Ben McDonald', 'Michael Waltner', 'Colleen Caulliez', "James O'Callaghan", 'Graham Coult', 'Damian Rogan', 'Jeffrey L. Robbin', 'Gregory N. Christie', 'Imran A. Chaudhri', 'Philip W. Schiller', 'Bas Ording', 'Patrick Coffman', 'Kenneth L. Kocienda', 'Nitin K. Ganatra', 'Freddy Allen Anzures', 'Jeremy A. Wyld', 'Jeffrey Alan Bush', 'Michael Matas', 'Paul D. Marcos', 'Charles J. Pisula', 'Scott Forstall',

## Exercise

Using the PatentsView API, let's get all patent descriptions for the patents that we found assigned to KCMO businesses.

In [186]:
business_patents = pd.read_csv('../../output/business_patents.csv')

/Users/nj995/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [187]:
patents = business_patents[pd.isnull(business_patents['patent_number'])==False]['patent_number']
patents = patents.drop_duplicates().tolist()

___ Query PatentsView API (200 patents at a time)___

This step can take up to 10 minutes.

In [161]:
df = pd.DataFrame("", index=np.arange(0), columns=['patent_abstract', 'patent_number', 'patent_title'])
start_time = time.time()
for i in range(0, int(math.modf(len(patents)/200)[1]+1)):
    sub_patents = patents[200*i:min(200*(i+1), len(patents))]
    base_url = 'http://www.patentsview.org/api/patents/query?'
    query = 'q={"_or":[{"patent_number":"'+ sub_patents[0] + '"}'
    for j in range(1, len(sub_patents)):
        query = query + ',{"patent_number":"'+sub_patents[j]+'"}'
    query = query + ']}'
    fields='&f=["patent_number", "patent_title", "patent_abstract"]'
    sort= '' #&s=
    options= '&o={"page":1,"per_page": 500}'
    url = '{}{}{}{}{}'.format(base_url, query, fields, sort, options)
    
    response = requests.get(url)
    response = json.loads(response.text)
    temp = DataFrame.from_dict(response['patents'])
    df = pd.concat([df, temp])

df = df[['patent_number', 'patent_title', 'patent_abstract']].sort_values('patent_number')
df = df.reset_index(drop = True)
print("--- %s seconds ---" % (time.time() - start_time))

--- 415.7428662776947 seconds ---


___ Merge back onto Business Names ___

In [195]:
patent_desc = pd.merge(business_patents[['business_name', 'patent_number']], df
                         , how = 'left', on = 'patent_number')
patent_desc = patent_desc[patent_desc['patent_number'].isnull()==False].drop_duplicates()

___ Export to CSV:___

In [196]:
patent_desc.to_csv('../../output/patent_desc.csv', index = False)

## Routing API

- Back to the [Table of Contents](#Table-of-Contents)

Now that we've tested we can access OTP from Jupyter, let's do something a bit more interesting: get a route plan between some locations. This will allow us to answer "How long will it take to get from *here* to *there*?"

Similar to the Index API, the [Routing API documentation](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html) tells us what features are available and how to access those feastures.

### Scraping Useful Chicago Data

- Back to the [Table of Contents](#Table-of-Contents)

To give us some valuable data to use with this API, I used the tools we used in our first lesson to build a quick scraper to get a lost of Chicago's workforce centers. You'll note that although the pattern of HTML differs, the structure of the scraper is very similar, and only requires a little bit of code.

In [ ]:
# Use requests to grab the HTML page for Chicago Workforce Centers
url = "http://deepdish.adrf.info/contrib/chicagojobs.html"
response = requests.get(url)

# Create BeautifulSoup object and pull out the table rows:
soup = BeautifulSoup(response.text)
table = soup.find("table")
rows = table.find_all("tr")

# Create lists to hold our scraped data
centers = []
addresses = []
phone_numbers = []

rows = rows[1:] #Skip the header row
for row in rows:
    
    name_td = row.find_all("td")[1]
    if name_td.find("a"):
        center_name = name_td.find("a").text
    else:        
        center_name = name_td.text

    centers.append(center_name)
    addresses.append(row.find_all("td")[2].text)
    phone_numbers.append(row.find_all("td")[3].text)
    
## Create pandas dataframe:
centers_df = pd.DataFrame({"center_name" : pd.Series(centers),
    "address" : pd.Series(addresses),
    "phone_number" : pd.Series(phone_numbers)})

## A little cleanup to remove extraneous tags:
centers_df["center_name"] = centers_df["center_name"].str.replace("<td>", "")
centers_df["center_name"] = centers_df["center_name"].str.replace("<br/>", "")

centers_df[:]

## Geocoding

- Back to the [Table of Contents](#Table-of-Contents)

We have scraped addresses, but OTP works best with latitude and longitude coordinates. We can use the [geocoder module](https://pypi.python.org/pypi/geocoder) to get latitude and longitude exactly just from the organization addresses. Note this is a combination of great tools - a simple Python module (`geocoder`) interacting with Google's wonderful geocoding API. The code below would work out in the wild, but since we are working in a restricted environment, we can't get to the Google API.

The geocoder module can speak to a wide range of external services, including ArcGIS, Bing, MapBox, OpenStreetMaps, and many others, in addition to Google. The API lets you geocode (addresses to latitue and longitude), reverse geocode (latitude and longitude to addresses), as well as get timezones and elevations on locations.

In [ ]:
# import geocoder
# 
# lat = []
# lon = []
#
# for add in centers_df["address"]:
#    g = geocoder.google(add)
    
#    lon.append(g.latlng[0])
#    lat.append(g.latlng[1])

# centers_df["latitude"] = pd.Series(lat)
# centers_df["longitude"] = pd.Series(lon)

In [ ]:
lat = [41.733737,
            41.9087846,
            41.9647695,
            41.9631174,
            42.0457523,
            41.9697109,
            41.9252578,
            41.4824241,
            41.5081785,
            41.8322347,
            41.6944193,
            41.8444394,
            41.8849173,
            41.5255653,
            41.8543913,
            41.8511856,
            41.8931701,
            41.8804296,
            41.8905965,
            42.0076194,
            41.9647485,
            41.8457521,
            41.8409604,
            41.9589605]

lon = [-87.770246,
             -87.7931388,
             -87.6786497,
             -87.6748518,
             -87.9922059,
             -87.6598793,
             -87.7008122,
             -87.6782855,
             -87.6234975,
             -87.5990999,
             -87.5990999,
             -87.7236882,
             -87.6231249,
             -87.6386009,
             -87.6355797,
             -87.7775432,
             -87.6614166,
             -87.7066519,
             -87.702801,
             -87.6689743,
             -87.6570292,
             -87.6858569,
             -87.6862319,
             -87.6747326]

In [ ]:
centers_lim = centers_df[0:24]
centers_lim["latitude"] = pd.Series(lat)
centers_lim["longitude"] = pd.Series(lon)
centers_lim[:]

### Planner Resource Syntax

- Back to the [Table of Contents](#Table-of-Contents)

The Planner Resource API does trip planning based on a large number of customizable parameters. To give you a sense of all the options available, OTP's planner resource allows users to set the additional time it will take baord a vehicle (like a bus) with a bike, as opposed to boarding on foot. There are a lot of available options. This is great for us, since once we understand the simple syntax of this API, we can avail ourselves of this granular customization if we want to. 

This resource is located at `/OTP/routers/{routerID}/plan` and when setting options within a URL, they follow a single question mark. So first, let's plan a trip with just the required options, `fromPlace`, `toPlace`, and `datae` (which takes an option in the form YYY-MM-DD) which you can see we set after `/plan?` and separated by ampersands `&`:

In [ ]:
origin_lat = centers_lim["latitude"][2]
origin_lon = centers_lim["longitude"][2]

destination_lat = centers_lim["latitude"][10]
destination_lon = centers_lim["longitude"][10]

qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2016-02-11'.format(base_url, origin_lat, origin_lon, destination_lat, destination_lon)   
print(qry_url)

response = requests.get(qry_url)
response = response.text
plan = json.loads(response)

# Examine the response, which is a routing plan:
print(plan)

In [ ]:
# So again our JSON object was transformed into a Python dict.
print(type(plan))

In [ ]:
# We can look at the available keys:
print(plan.keys())

In [ ]:
# And use those keys to see the dict'svalues.
# For instance, let's print out the requestParameters:
print(plan['requestParameters'])

In [ ]:
# Dicts can contain other dicts, like in the case of the plan:
print(type(plan["plan"]))
print(plan['plan'].keys())

In [ ]:
## We can use a similar syntax torfer to the keys of a dict within a dict:
print(plan["plan"]["to"])

In [ ]:
# Time is stored in a raw computer format
print('raw time value: {}'.format(plan['plan']['date']))

# But we can convert it to a datetime object so it's comprehensible.
# note OTP returns raw time value with three extra zeros, divide by 1000 to get rid of them
print('datetime formatted: {}'.format(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(plan['plan']['date']/1000))))

In [ ]:
# 'itineraries' holds a lot more information, let's start with how many itineraries were returned
print(len(plan['plan']['itineraries']))

# and list what keys exit for the first itinerary
print(plan['plan']['itineraries'][0].keys())

In [ ]:
# compare the three itineraries across some pieces
for i in plan['plan']['itineraries']:
    print('duration (minutes) = {0:.2f} | transfers = {1:} | walkDist = {2:.2f} | \
legs = {4:} | startTime = {5:} | endTime = {6:}'\
.format(i['duration']/60., i['transfers'], i['walkDistance'], i['walkTime'], len(i['legs']), 
        time.strftime('%H:%M:%S', time.localtime(i['startTime']/1000)), 
        time.strftime('%H:%M:%S', time.localtime(i['endTime']/1000))))

In [ ]:
# note we just counted the length of the "legs" output, it contains the details of the actual route
# here is what is included in a "leg"
print(plan['plan']['itineraries'][0]['legs'][0].keys())

In [ ]:
# let's compare the three legs of the first itinerary, similarly as we compared the itineraries
for leg in plan['plan']['itineraries'][0]['legs']:
    print('distance = {:,.2f} | duration = {:.0f} | mode = {} | route = {} | steps = {}'.\
format(leg['distance'], leg['duration'], leg['mode'], leg['route'], len(leg['steps'])))

So, if mode is 'WALK' then route is blank and steps is a list. what is included in one of those 'steps'?

In [ ]:
print(plan['plan']['itineraries'][0]['legs'][0]['steps'][0].keys())

In [ ]:
# so, what streets does this first route call for a person to walk on?
for leg in plan['plan']['itineraries'][0]['legs']:
    print('leg sends person on following streets:')
    if leg['mode']=='WALK':
        for step in leg['steps']:
            print(step['streetName'])
    else:
        print('N/A - not a walking leg.')

## Adding Further Options:

The arriveBy parameter takes a time hh:mm:ss - can you add this option to our query?

What happens if you increase the bikeSpeed option (takes an integer in MPH - defaults to 11 MPH)?

In [ ]:
your_qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2016-06-05'.format(base_url, origin_lat, origin_lon, destination_lat, destination_lon)   
print(your_qry_url)

response = requests.get(your_qry_url)
response = response.text
plan = json.loads(response)

print(plan)

## Incorporating Databases

- Back to the [Table of Contents](#Table-of-Contents)

We can use SQL Alchemy to grab some data from our databases.

In [ ]:
# Database connection
from sqlalchemy import create_engine
engine = create_engine("postgresql://10.10.2.10:5432/appliedda")

In [ ]:
pd.read_sql("SELECT table_name FROM information_schema.tables;", engine)

In [ ]:
# The tl_2016_16980_tabblock10 table contains census blocks for Chicago:
blocks_df = pd.read_sql("SELECT * FROM tl_2016_16980_tabblock10;", engine)
blocks_df[:5]

In [ ]:
# Block on East Side of Douglas Park
## Used City of Chicago Data Portal to find the geoid10:
block = blocks_df[blocks_df["geoid10"] == "170318433001046"]
block[:]

In [ ]:
centers_lim[:5]

In [ ]:
block_lat = 41.8599228
block_lon = -87.6944049

for i in range(0,len(centers_lim.index)):
    
    center = centers_lim.center_name[i]
    center_lat = centers_lim.latitude[i]
    center_lon = centers_lim.longitude[i]
    
    print(center)
    
    qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2016-06-10%mode=WALK,TRANSIT'.format(
        base_url, block_lat, block_lon, center_lat, center_lon)   
    
    try:
        response = requests.get(qry_url).text
        plan = json.loads(response)
        print(plan)
                
    except requests.exceptions.RequestException:
        pass
    

In [ ]:
centers_lim.count()

In [ ]:
origin_lat = centers_lim["latitude"][2]
origin_lon = centers_lim["longitude"][2]

destination_lat = centers_lim["latitude"][10]
destination_lon = centers_lim["longitude"][10]

qry_url = '{}plan?fromPlace={},{}&toPlace={},{}&date=2016-02-11'.format(base_url, origin_lat, origin_lon, destination_lat, destination_lon)   
print(qry_url)

response = requests.get(qry_url)
response = response.text
plan = json.loads(response)

# Examine the response, which is a routing plan:
print(plan)

## Isochrone API

- Back to the [Table of Contents](#Table-of-Contents)

The Isochrone (meaning same-time) tool gives the area (as a polygon) a traveler can reach from a specified point within a travel time. Like the other APIs, the Isochrone API has many other query parameters the user can set if so desired, [description here](http://dev.opentripplanner.org/apidoc/1.0.0/resource_LIsochrone.html). It requires that we define a starting location, a mode of transportation, a date, and an amount of travel time.

Below, we start in downtown Chicago, allowing use of foot and public transit, on a certain date, and with 30 minutes of travel time allowed.

In [ ]:
# set start location
start_point = [41.846698, -87.621385] # Mercy Hospital & Medical Center

travel_time = 60 * 30 # time in seconds, so this is 30 minutes
mode = "WALK,TRANSIT"

url = ("{}isochrone?fromPlace={},{}&mode={}&date=2016-06-01&cutoffSec={}").format(
    base_url,start_point[0],start_point[1],mode,travel_time)
print(url)

iso_response = requests.get(url)
print(iso_response.text)

> NOTE: as of Nov 1 underlying packages for spatial Python functions need to be updated on ADRF so the below will not work

In [ ]:
iso_json = json.loads(iso_response.text)

## load isochrone into a geopandas dataframe
iso_gdf = gpd.GeoDataFrame.from_features(iso_json['features'])
iso_gdf[:]

In [ ]:
# view the resulting isochrone shape (can you guess why there are separated geographies?)
iso_gdf.plot();

One potential use case for this functionality: can people at two locations reach some common location within a specified travel time?

In [ ]:
# 2nd location
start_point_2 = [41.884260, -87.630344] # Traffic Court in Richard J. Daley center

url_2 = ("{}isochrone?fromPlace={},{}&mode={}&date=2016-06-01&cutoffSec={}").format(
    base_url,start_point_2[0],start_point[1],mode,travel_time)

# get json request
iso_json_2 = json.loads(requests.get(url_2).text)

## load isochrone into a geopandas dataframe
iso_gdf_2 = gpd.GeoDataFrame.from_features(iso_json_2['features'])

In [ ]:
# view the second isochrone
iso_gdf_2.plot();

In [ ]:
# do the two isochrones intersect?
iso_gdf.intersects(iso_gdf_2)

In [ ]:
# they do intersect, so create an overlay with a 'union'
iso_join = overlay(iso_gdf, iso_gdf_2, how='union')

In [ ]:
# what does the dataframe look like now?
iso_join.head()

In [ ]:
# and visually?
iso_join.plot();

- Back to the [Table of Contents](#Table-of-Contents)

> A bit annoyingly this is difficult to tell where the two overlap. To fix this we can group based on the "time" and "time_2" columns to end with just 3 combinations: 
1. accessible from our first location only, 
2. accessible our second location only, and 
3. accessible from either location

We'll do this by using the ["dissolve" function](http://geopandas.org/aggregation_with_dissolve.html) from geopandas. However first we need to replace the "NaN" so those rows are not ignored

In [ ]:
# replace NaN with placeholder value, let's say 99999
iso_join.fillna(99999, inplace=True)

In [ ]:
iso_join = iso_join.dissolve(by=['time', 'time_2']).reset_index()

# Note: used reset_index() here so it's easier to use the 'time' and 'time_2' columns if needed

In [ ]:
# now what does it look like?
iso_join

In [ ]:
# and visually?
iso_join.plot();

In [ ]:
# add a label column to use so we can include a legend
iso_join['label'] = ''

# use index slicing function '.loc' of dataframes to update each value of label appropriately
iso_join.loc[0,'label'] = 'Both'
iso_join.loc[1,'label'] = 'point 1 only'
iso_join.loc[2,'label'] = 'point 2 only'

In [ ]:
# set up a nicer visualization with labels
f,ax = plt.subplots(figsize=(8,8))

# use geopandas to specify label column and adding a legend to the matplotlib object 'ax'
iso_join.plot(column='label', ax=ax, legend=True);

# also plot start and stop points on the same map, note matplotlib takes [x,y] coordinates
ax.plot(start_point[1], start_point[0], 's', color='orange', markersize=10, label='point 1')
ax.plot(start_point_2[1], start_point_2[0], 's', color='grey', markersize=10, label='point 2')

# add some other labels
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# title
ax.set_title('Area accessible within {} minutes travel using public transit'.format(travel_time/60));